In [19]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline

from tyssue.core.sheet import Sheet

from tyssue import config
from tyssue.topology.sheet_topology import cell_division

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.solvers.sheet_vertex_solver import Solver as solver
from tyssue.dynamics.sheet_vertex_model import SheetModel as model

from tyssue.core.generation import extrude
from tyssue.core.monolayer import Monolayer, MonolayerWithLamina
from tyssue.geometry.bulk_geometry import BulkGeometry, MonoLayerGeometry
from tyssue.dynamics.bulk_model import BulkModel, LaminaModel, set_model
from tyssue.solvers.sheet_vertex_solver import Solver

from tyssue.draw.vispy_draw import vp_view

from tyssue.draw.plt_draw import sheet_view

from tyssue.topology.sheet_topology import remove_face
from tyssue.core.objects import get_opposite
from tyssue.io.hdf5 import save_datasets, load_datasets

from tyssue.topology.sheet_topology import face_division, get_division_edges


In [20]:
datasets = load_datasets('small_ellipsoid.hf5',
                         data_names=['vert', 'edge',
                                     'face', 'cell'])
specs = config.geometry.bulk_spec()

mono = Monolayer('ell', datasets, specs)
mono.reset_index()
mono.reset_topo()


# ## Settings for the quasi-static model
# ### Apical layer
apical_spec = {
    'edge': {
        'line_tension': 0.12,},
    'face': {
        'contractility': 0.04,
        'prefered_area': 0.8,
        'area_elasticity': 1.0},
    'cell': {
        'prefered_vol': 1.0,
        'vol_elasticity': 1.0},
    'settings': {
        'grad_norm_factor': 1.0,
        'nrj_norm_factor': 1.0},}

# ### Modifier for each segment of the stratum
modifiers = {
    'sagittal': {
        'edge': {
            'line_tension': 0.1},
        'face': {
            'contractility': 0.05,
            'prefered_area': 0.2}
    },
    'basal': {
        'edge': {
            'line_tension': 0.5},
        'face': {
            'contractility': 0.1,
            'prefered_area': 0.2}},
    }

set_model(mono, BulkModel, apical_spec, modifiers)





In [21]:
draw_specs = config.draw.sheet_spec()
canvas, view = vp_view(mono, **draw_specs)

In [22]:
res = Solver.find_energy_min(mono, BulkGeometry, BulkModel)

In [23]:
canvas, view = vp_view(mono)
monolayer = mono

In [24]:
mother = 15
mother_edges = monolayer.edge_df[
    monolayer.edge_df['cell'] == mother]
mother_faces = set(mother_edges['face'])

apical_face, = mother_edges[
    mother_edges['segment'] == 'apical']['face'].unique()
basal_face, = mother_edges[
    mother_edges['segment'] == 'basal']['face'].unique()


angle = np.random.random() * np.pi
edge_aa, edge_ab = get_division_edges(monolayer, apical_face,
                                      BulkGeometry,
                                      angle=angle, axis='x')
edge_ba, edge_bb = get_division_edges(monolayer, basal_face,
                                      BulkGeometry,
                                      angle=angle-np.pi, axis='x')

srce_aa, trgt_aa = mono.edge_df.loc[edge_aa, ['srce', 'trgt']]
srce_ab, trgt_ab = mono.edge_df.loc[edge_ab, ['srce', 'trgt']]

srce_ba, trgt_ba = mono.edge_df.loc[edge_ba, ['srce', 'trgt']]
srce_bb, trgt_bb = mono.edge_df.loc[edge_bb, ['srce', 'trgt']]


def get_edges(edge_df, srce, trgt):
    edges = edge_df[(edge_df['srce'] == srce) &
                    (edge_df['trgt'] == trgt)]
    if edges.shape[0] == 1:
        return edges.index[0]
    else:
        return edges.index.values


In [25]:
edges_aa = get_edges(monolayer.edge_df,
                     srce_aa, trgt_aa)
opp_edges_aa = get_edges(monolayer.edge_df,
                        trgt_aa, srce_aa)

edges_ab = get_edges(monolayer.edge_df,
                     srce_ab, trgt_ab)
opp_edges_ab = get_edges(monolayer.edge_df,
                         trgt_ab, srce_ab)

edges_ba = get_edges(monolayer.edge_df,
                     srce_ba, trgt_ba)
opp_edges_ba = get_edges(monolayer.edge_df,
                        trgt_ba, srce_ba)

edges_bb = get_edges(monolayer.edge_df,
                     srce_bb, trgt_bb)
opp_edges_bb = get_edges(monolayer.edge_df,
                         trgt_bb, srce_bb)

all_edges = np.concatenate([edges_aa, opp_edges_aa,
                            edges_ab, opp_edges_ab,
                            edges_ba, opp_edges_ba,
                            edges_bb, opp_edges_bb])
print(all_edges)

[  59  804   63  788  265 1608  264 1612  335  806  339  790  541 1610  540
 1614]


In [26]:
old_nf = monolayer.Nf

In [27]:
monolayer.edge_df.loc[all_edges].sort_values(['face'])

,srce,trgt,face,segment,cell,nz,dy,dx,ny,nx,dz,sub_area,length,sub_vol,prefered_length,length_eleasticity,line_tension
edge,,,,,,,,,,,,,,,,,
59,24,19,15,apical,15,-0.052023,0.133117,0.245382,0.071283,-0.004489,0.161229,0.044181,0.322377,0.003418,0.0,0.0,0.120
265,66,88,15,apical,15,-0.046537,-0.135455,-0.200833,0.048334,-0.010480,-0.095457,0.033955,0.260373,0.002624,0.0,0.0,0.120
63,19,24,16,apical,16,-0.019217,-0.133117,-0.245382,0.079843,-0.030687,-0.161229,0.043834,0.322377,0.003060,0.0,0.0,0.120
264,88,66,56,apical,56,-0.052075,0.135455,0.200833,0.008327,0.019136,0.095457,0.028051,0.260373,0.002396,0.0,0.0,0.120
335,111,116,79,basal,15,0.011870,-0.066745,-0.122334,-0.014136,-0.000230,-0.081861,0.009230,0.161622,0.000726,0.0,0.0,0.060
541,180,158,79,basal,15,0.009528,0.067065,0.111775,-0.010525,0.000327,0.070247,0.007101,0.148074,0.000559,0.0,0.0,0.060
339,116,111,80,basal,16,0.006436,0.066745,0.122334,-0.023583,0.008560,0.081861,0.012951,0.161622,0.000908,0.0,0.0,0.060
540,158,180,120,basal,56,0.005455,-0.067065,-0.111775,-0.001034,-0.002808,-0.070247,0.003111,0.148074,0.000266,0.0,0.0,0.060
788,19,24,187,sagittal,15,0.051372,-0.133117,-0.245382,0.024124,-0.046841,-0.161229,0.036794,0.322377,0.002355,0.0,0.0,0.012


In [28]:
daughters = []

for face, df in monolayer.edge_df.loc[all_edges].groupby('face'):
    if df.shape[0] == 2:
        print(df.index.values)
        edge_a, edge_b = df.index.values
        cell = df['cell'].iloc[0]
        daughter = face_division(monolayer, edge_a, edge_b)
        
        if cell == mother:
            daughters.append(daughter)
        
        

[ 59 265]
[335 541]
[788 790]
[804 806]
[1608 1610]
[1612 1614]


In [28]:
%pdb

Automatic pdb calling has been turned ON


In [29]:
canvas, view = vp_view(mono)

In [52]:
vert_orbit = monolayer.get_orbits('face', 'srce').groupby(
    level='face').apply(lambda df: set(df))
